In [75]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn.model_selection import train_test_split


# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))
        
data = pd.read_csv("/kaggle/input/cardiovascular-disease-dataset/Cardiovascular_Disease_Dataset/Cardiovascular_Disease_Dataset.csv")
data.rename(columns={'gender': 'male'}, inplace=True)
print(data["restingBP"].min())
print(data["restingBP"].max())
# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session
data.drop(columns=["patientid"], axis=1, inplace=True)
print(data["male"].value_counts())
print(data["target"].value_counts())
print(data["target"][data["male"]==0].value_counts())
X = data.drop('target', axis=1)
y = data['target']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
data

/kaggle/input/cardiovascular-disease-dataset/Cardiovascular_Disease_Dataset/Cardiovascular_Disease_Dataset.csv
/kaggle/input/cardiovascular-disease-dataset/Cardiovascular_Disease_Dataset/Cardiovascular_Disease_Dataset_Description.pdf
94
200
male
1    765
0    235
Name: count, dtype: int64
target
1    580
0    420
Name: count, dtype: int64
target
1    133
0    102
Name: count, dtype: int64


,age,male,chestpain,restingBP,serumcholestrol,fastingbloodsugar,restingrelectro,maxheartrate,exerciseangia,oldpeak,slope,noofmajorvessels,target
0,53,1,2,171,0,0,1,147,0,5.3,3,3,1
1,40,1,0,94,229,0,1,115,0,3.7,1,1,0
2,49,1,2,133,142,0,0,202,1,5.0,1,0,0
3,43,1,0,138,295,1,1,153,0,3.2,2,2,1
4,31,1,1,199,0,0,2,136,0,5.3,3,2,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,48,1,2,139,349,0,2,183,1,5.6,2,2,1
996,47,1,3,143,258,1,1,98,1,5.7,1,0,0
997,69,1,0,156,434,1,0,196,0,1.4,3,1,1
998,45,1,1,186,417,0,1,117,1,5.9,3,2,1


In [76]:
#KNeighbours
from sklearn.model_selection import GridSearchCV
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score
param_grid = {'n_neighbors': [5, 7, 11, 15, 21, 35, 31, 29, 27]}
knn = KNeighborsClassifier()
scoring_metrics = ['accuracy', 'precision']
grid_search = GridSearchCV(knn, param_grid, scoring=scoring_metrics, refit='precision', cv=5)
grid_search.fit(X_train, y_train)
best_k = grid_search.best_params_['n_neighbors']
best_knn = KNeighborsClassifier(n_neighbors=best_k)
best_knn.fit(X_train, y_train)
y_pred = best_knn.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
print(f'Best k: {best_k}')
print(f'Accuracy: {accuracy:.4f}')
print(f'Precision: {precision:.4f}')
print(f'Recall: {recall:.4f}')

Best k: 29
Accuracy: 0.8300
Precision: 0.8430
Recall: 0.8718


In [77]:
#DecisionTreeClassifier
from sklearn.tree import DecisionTreeClassifier

param_grid = {
    'random_state': list(range(10)),
    'max_depth': [None, 10, 20],
    'min_samples_leaf': [1, 5, 10],
    'criterion': ['gini', 'entropy'],
}
tree_classifier = DecisionTreeClassifier()
scoring_metrics = ['accuracy', 'precision', 'recall']
grid_search = GridSearchCV(tree_classifier, param_grid, scoring=scoring_metrics, refit='recall', cv=5)
grid_search.fit(X_train, y_train)
best_params = grid_search.best_params_
best_tree_classifier = DecisionTreeClassifier(**best_params)
best_tree_classifier.fit(X_train, y_train)
y_pred = best_tree_classifier.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
print(f'Best parameters: {best_params}')
print(f'Accuracy: {accuracy:.4f}')
print(f'Precision: {precision:.4f}')
print(f'Recall: {recall:.4f}')

Best parameters: {'criterion': 'entropy', 'max_depth': None, 'min_samples_leaf': 1, 'random_state': 1}
Accuracy: 0.9500
Precision: 0.9652
Recall: 0.9487


In [78]:
#RandomForestClassifier
from sklearn.ensemble import RandomForestClassifier

param_grid = {
    'random_state': list(range(10)), 
    'max_depth': [None, 10, 20],
    'min_samples_leaf': [1, 5, 10],
    'criterion': ['gini', 'entropy'],
}
tree_classifier = DecisionTreeClassifier()
scoring_metrics = ['accuracy', 'precision', 'recall']
grid_search = GridSearchCV(tree_classifier, param_grid, scoring=scoring_metrics, refit='recall', cv=5)
grid_search.fit(X_train, y_train)
best_params = grid_search.best_params_
best_tree_classifier = DecisionTreeClassifier(**best_params)
best_tree_classifier.fit(X_train, y_train)
y_pred = best_tree_classifier.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
print(f'Best parameters: {best_params}')
print(f'Accuracy: {accuracy:.4f}')
print(f'Precision: {precision:.4f}')
print(f'Recall: {recall:.4f}')

Best parameters: {'criterion': 'entropy', 'max_depth': None, 'min_samples_leaf': 1, 'random_state': 1}
Accuracy: 0.9500
Precision: 0.9652
Recall: 0.9487


In [79]:
from sklearn.metrics import classification_report
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
def create_nn_model(optimizer='adam', hidden_units=64):
    model = Sequential()
    model.add(Dense(hidden_units, input_dim=X_train.shape[1], activation='relu'))
    model.add(Dense(1, activation='sigmoid'))
    model.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])
    return model
nn_model = Sequential()
nn_model.add(Dense(64, input_dim=X_train.shape[1], activation='relu'))
nn_model.add(Dense(1, activation='sigmoid'))
nn_model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
nn_model.fit(X_train, y_train, epochs=29, batch_size=25, verbose=0)
y_pred_prob = nn_model.predict(X_test)
y_pred = (y_pred_prob > 0.5).astype(int)
accuracy = accuracy_score(y_test, y_pred)
classification_rep = classification_report(y_test, y_pred)
print(f"Accuracy on Test Set: {accuracy:.4f}")
print("Classification Report:\n", classification_rep)


7/7 [==============================] - 0s 2ms/step
Accuracy on Test Set: 0.8900
Classification Report:
               precision    recall  f1-score   support

           0       1.00      0.73      0.85        83
           1       0.84      1.00      0.91       117

    accuracy                           0.89       200
   macro avg       0.92      0.87      0.88       200
weighted avg       0.91      0.89      0.89       200



In [80]:
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)
svm_model = SVC()
param_grid = {'C': [0.1, 1, 10],
              'kernel': ['linear', 'rbf']}
grid_search = GridSearchCV(svm_model, param_grid, cv=5, scoring='accuracy')
grid_search.fit(X_train_scaled, y_train)
best_params = grid_search.best_params_
best_svm_model = SVC(**best_params)
best_svm_model.fit(X_train_scaled, y_train)
y_pred = best_svm_model.predict(X_test_scaled)
accuracy = accuracy_score(y_test, y_pred)
classification_rep = classification_report(y_test, y_pred)

print(f"Best Hyperparameters: {best_params}")
print(f"Accuracy on Test Set: {accuracy:.4f}")
print("Classification Report:\n", classification_rep)

Best Hyperparameters: {'C': 10, 'kernel': 'rbf'}
Accuracy on Test Set: 0.9750
Classification Report:
               precision    recall  f1-score   support

           0       0.98      0.96      0.97        83
           1       0.97      0.98      0.98       117

    accuracy                           0.97       200
   macro avg       0.98      0.97      0.97       200
weighted avg       0.98      0.97      0.97       200



In [81]:
from sklearn.ensemble import VotingClassifier
from xgboost import XGBClassifier
from sklearn.model_selection import cross_val_score, StratifiedKFold
from sklearn.metrics import make_scorer

def create_xgboost_model(learning_rate=0.1, max_depth=2, n_estimators=129):
    model = XGBClassifier(learning_rate=learning_rate, max_depth=max_depth, n_estimators=n_estimators)
    return model

def create_svm_model(C=10, kernel='rbf'):
    model = SVC(C=C, kernel=kernel, probability=True)
    return model

xgboost_model = create_xgboost_model()
svm_model = create_svm_model()

voting_model = VotingClassifier(estimators=[('xgboost', xgboost_model), ('svm', svm_model)], voting='soft')

cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=35)
cv_scores_voting = cross_val_score(voting_model, X_train, y_train, cv=cv, scoring=make_scorer(accuracy_score))

print("Voting Classifier Cross-Validation Scores:", cv_scores_voting)
print("Mean Accuracy (Voting Classifier): {:.4f}".format(cv_scores_voting.mean()))

voting_model.fit(X_train, y_train)

y_pred_voting = voting_model.predict(X_test)

accuracy_voting = accuracy_score(y_test, y_pred_voting)
classification_rep_voting = classification_report(y_test, y_pred_voting)

print(f"\nAccuracy on Test Set (Voting Classifier): {accuracy_voting:.4f}")
print("Classification Report:\n", classification_rep_voting)


Voting Classifier Cross-Validation Scores: [0.95    0.95625 0.99375 0.96875 0.96875]
Mean Accuracy (Voting Classifier): 0.9675

Accuracy on Test Set (Voting Classifier): 0.9700
Classification Report:
               precision    recall  f1-score   support

           0       0.96      0.96      0.96        83
           1       0.97      0.97      0.97       117

    accuracy                           0.97       200
   macro avg       0.97      0.97      0.97       200
weighted avg       0.97      0.97      0.97       200



In [82]:
from sklearn.model_selection import cross_val_score, StratifiedKFold
from xgboost import XGBClassifier
from sklearn.metrics import make_scorer

def create_xgboost_model(learning_rate=0.1, max_depth=2, n_estimators=129):
    model = XGBClassifier(learning_rate=learning_rate, max_depth=max_depth, n_estimators=n_estimators)
    return model
xgboost_model = create_xgboost_model()
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=35)
cv_scores = cross_val_score(xgboost_model, X_train, y_train, cv=cv, scoring=make_scorer(accuracy_score))
print("Cross-Validation Scores:", cv_scores)
print("Mean Accuracy: {:.4f}".format(cv_scores.mean()))
xgboost_model.fit(X_train, y_train)
y_pred = xgboost_model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
classification_rep = classification_report(y_test, y_pred)

print(f"\nAccuracy on Test Set: {accuracy:.4f}")
print("Classification Report:\n", classification_rep)
print(xgboost_model.feature_importances_)

#Cross-Validation Scores: [0.975   0.975   0.9875  0.98125 0.9625 ]
#Mean Accuracy: 0.9763
#
#Accuracy on Test Set: 0.9900
#Classification Report:
#               precision    recall  f1-score   support
#
 #          0       1.00      0.98      0.99        83
#           1       0.98      1.00      0.99       117
#
#    accuracy                           0.99       200
#   macro avg       0.99      0.99      0.99       200
#weighted avg       0.99      0.99      0.99       200
#
#[0.01262199 0.02088706 0.09115338 0.09086297 0.05000516 0.03869672
 #0.05893775 0.0352497  0.         0.04109602 0.509956   0.05053324]

Cross-Validation Scores: [0.975   0.975   0.9875  0.98125 0.9625 ]
Mean Accuracy: 0.9763

Accuracy on Test Set: 0.9900
Classification Report:
               precision    recall  f1-score   support

           0       1.00      0.98      0.99        83
           1       0.98      1.00      0.99       117

    accuracy                           0.99       200
   macro avg       0.99      0.99      0.99       200
weighted avg       0.99      0.99      0.99       200

[0.01262199 0.02088706 0.09115338 0.09086297 0.05000516 0.03869672
 0.05893775 0.0352497  0.         0.04109602 0.509956   0.05053324]


In [83]:
datar = data.copy()
datar['chestpain'] = (data['chestpain'] > 0).astype(int)
selected_columns = ['serumcholestrol', 'fastingbloodsugar', 'restingrelectro', 'oldpeak', 'slope', 'noofmajorvessels']
datar[selected_columns] = datar[selected_columns].mean()
X = datar.drop('target', axis=1)
y = datar['target']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
y_test

521    1
737    0
740    0
660    0
411    1
      ..
408    1
332    0
208    0
613    1
78     1
Name: target, Length: 200, dtype: int64

In [84]:
from sklearn.model_selection import cross_val_score, StratifiedKFold
from xgboost import XGBClassifier
from sklearn.metrics import make_scorer

def create_xgboost_model(learning_rate=0.1, max_depth=2, n_estimators=129):
    model = XGBClassifier(learning_rate=learning_rate, max_depth=max_depth, n_estimators=n_estimators)
    return model
xgboost_model = create_xgboost_model()
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=35)
cv_scores = cross_val_score(xgboost_model, X_train, y_train, cv=cv, scoring=make_scorer(accuracy_score))
print("Cross-Validation Scores:", cv_scores)
print("Mean Accuracy: {:.4f}".format(cv_scores.mean()))
xgboost_model.fit(X_train, y_train)
y_probabilities = xgboost_model.predict_proba(X_test)
custom_threshold = 0.3
y_predictions = (y_probabilities > custom_threshold)[:, 1].astype(int)
print(y_predictions)

# Evaluate the model with the chosen threshold
accuracy = accuracy_score(y_test, y_predictions)
classification_rep = classification_report(y_test, y_predictions)

print(f"Accuracy with custom threshold {custom_threshold}: {accuracy:.4f}")
print("Classification Report:\n", classification_rep)

Cross-Validation Scores: [0.88125 0.875   0.9     0.8625  0.91875]
Mean Accuracy: 0.8875
[1 1 1 1 1 0 0 1 1 1 0 1 1 1 1 1 1 1 0 1 1 1 0 0 0 1 0 1 0 1 1 0 1 1 1 1 1
 1 0 0 0 1 0 1 0 1 0 1 0 1 0 1 1 0 1 0 1 1 1 1 0 1 0 1 0 0 1 1 1 1 0 1 1 1
 0 1 1 0 1 0 1 1 0 1 0 1 1 0 0 1 0 1 0 0 1 1 1 0 0 0 0 1 0 1 1 1 0 1 1 1 1
 0 1 0 1 1 0 0 0 0 1 0 0 1 0 1 1 1 0 1 1 1 1 1 1 1 1 1 1 1 1 0 1 1 0 1 1 0
 1 1 0 0 0 1 1 1 1 1 1 1 1 0 1 0 1 0 1 1 0 0 1 1 1 0 0 1 0 1 0 1 0 1 0 1 0
 1 1 1 1 0 0 1 1 1 1 1 0 1 1 1]
Accuracy with custom threshold 0.3: 0.8950
Classification Report:
               precision    recall  f1-score   support

           0       0.93      0.81      0.86        83
           1       0.88      0.96      0.91       117

    accuracy                           0.90       200
   macro avg       0.90      0.88      0.89       200
weighted avg       0.90      0.90      0.89       200

